## 262. Trips and Users

```
The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Write a SQL query to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03". Round Cancellation Rate to two decimal points.

Return the result table in any order.
```

In [1]:
import os
import sys

import pandas as pd
import pandasql as ps

_path = os.path.abspath(os.path.join('..'))
sys.path.append(_path)
from util.parser import parse_table

In [2]:
t = '''
+----+-----------+-----------+---------+---------------------+------------+
| id | client_id | driver_id | city_id | status              | request_at |
+----+-----------+-----------+---------+---------------------+------------+
| 1  | 1         | 10        | 1       | completed           | 2013-10-01 |
| 2  | 2         | 11        | 1       | cancelled_by_driver | 2013-10-01 |
| 3  | 3         | 12        | 6       | completed           | 2013-10-01 |
| 4  | 4         | 13        | 6       | cancelled_by_client | 2013-10-01 |
| 5  | 1         | 10        | 1       | completed           | 2013-10-02 |
| 6  | 2         | 11        | 6       | completed           | 2013-10-02 |
| 7  | 3         | 12        | 6       | completed           | 2013-10-02 |
| 8  | 2         | 12        | 12      | completed           | 2013-10-03 |
| 9  | 3         | 10        | 12      | completed           | 2013-10-03 |
| 10 | 4         | 13        | 12      | cancelled_by_driver | 2013-10-03 |
+----+-----------+-----------+---------+---------------------+------------+
'''

u = '''
+----------+--------+--------+
| users_id | banned | role   |
+----------+--------+--------+
| 1        | No     | client |
| 2        | Yes    | client |
| 3        | No     | client |
| 4        | No     | client |
| 10       | No     | driver |
| 12       | No     | driver |
| 11       | No     | driver |
| 13       | No     | driver |
+----------+--------+--------+
'''

In [39]:
query = '''
--build a lookup table first
with CTE as (
    select t.*, c.client_banned, d.driver_banned
    from Trips t
    inner join (
        select 
            users_id, 
            banned as client_banned
        from Users
    ) c
    on c.users_id = t.client_id
    inner join (
        select 
            users_id,
            banned as driver_banned
        from Users
    ) d
    on d.users_id = t.driver_id
)
select 
    request_at as Day,
    ROUND(AVG(cancelled), 2) as 'Cancellation Rate' 
from (
    select 
        *, 
        CASE WHEN (status != 'completed') THEN 1 ELSE 0 END as 'cancelled'
    from CTE
    where client_banned = 'No' and driver_banned = 'No'
) res
group by 
    request_at
'''

In [40]:
Trips = parse_table(t)
Users = parse_table(u)

In [41]:
ps.sqldf(query, locals())

,Day,Cancellation Rate
0,2013-10-01,0.0
1,2013-10-02,0.0
2,2013-10-03,1.0


In [21]:
Trips

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [4]:
query = '''
select request_at as Day, ROUND(AVG(counter), 2) as 'Cancellation Rate'
from (
    select
        u1.*,
        CASE WHEN (u1.status != 'completed') THEN 1 ELSE 0 END as counter
    from (
        select t.*, u.user_banned
        from Trips t
        inner join (
            select users_id, banned as user_banned
            from Users
        ) u
        on t.client_id = u.users_id
    ) u1
    inner join (
        select users_id, banned as driver_banned
        from Users
    ) u2
    on u1.driver_id = u2.users_id
    where u1.user_banned = 'No' and u2.driver_banned = 'No'
) res
group by request_at
'''

In [5]:
ps.sqldf(query, locals())

,Day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


In [6]:
Trips = pd.DataFrame([], columns=['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at'])

In [7]:
Trips

,id,client_id,driver_id,city_id,status,request_at


In [8]:
ps.sqldf(query, locals())

,Day,Cancellation Rate


In [9]:
import sqlite3

In [10]:
conn = sqlite3.connect('test.sql')

In [11]:
Users.to_sql('Users', conn)
Trips.to_sql('Trips', conn)

0

In [12]:
pd.read_sql_query(query, conn)

,Day,Cancellation Rate
